In [1]:
import pprint
from ipywidgets import widgets
import getpass
import ccqclient
import pprint
import urllib
from ipywidgets import widgets

In [2]:

#login_instance_url='login1-zeolitesalmoncod.cloudycluster.net'
login_instance_url='login-whaleemeraldbetta.cloudycluster.net'
# user_name = input ('username')
user_name = 'admin'
user_pass = getpass.getpass()
# user_pass = 'xxxxxxxx'
client = ccqclient.CCQClient(login_instance_url, user_name, user_pass, ccqclient.CCQCloud.GCP, ccqclient.CCQScheduler.Slurm)

 ········


In [3]:
ccq_menu = widgets.Dropdown(
       options=['ccqstat', 'ccqsub', 'ccqdel'],
       value='ccqsub',
       description='command:')
display(ccq_menu)

Dropdown(description='command:', index=1, options=('ccqstat', 'ccqsub', 'ccqdel'), value='ccqsub')

In [4]:
params = {
  'num_nodes' : 0,
  'num_tasks_per_node' : 0,
    'file_path' : '',
    'mpi_compile' : '',
    'mpi_type' : ''
}

In [5]:
num_nodes = widgets.IntText(
    value = 2,
    description='num_nodes:',
    disabled=False)
display(num_nodes)

num_tasks_per_node =  widgets.IntText(
    value = 2,
    description='num_tasks_per_node:',
    disabled=False)
display(num_tasks_per_node)

file_path = widgets.Text(
    value='/mnt/orangefs/samplejobs/mpi/',
    placeholder='Enter path',
    description='Path:',
    disabled=False
)
display(file_path)

IntText(value=2, description='num_nodes:')

IntText(value=2, description='num_tasks_per_node:')

Text(value='/mnt/orangefs/samplejobs/mpi/', description='Path:', placeholder='Enter path')

In [6]:
mpi_compile =  widgets.Dropdown(
   options=['mpicc','mpifort','mpicxx','mpif77'],
   value='mpicc',
   description='Compile as:')
display(mpi_compile)

mpi_type = widgets.Dropdown(
   options=['mpirun','mpiexec'],
   value='mpirun',
   description='Run using:')
display(mpi_type)

Dropdown(description='Compile as:', options=('mpicc', 'mpifort', 'mpicxx', 'mpif77'), value='mpicc')

Dropdown(description='Run using:', options=('mpirun', 'mpiexec'), value='mpirun')

In [7]:
def createScript(params_struct):
    params_struct[num_nodes] = num_nodes.value
    params_struct[num_tasks_per_node] = num_tasks_per_node.value
    params_struct[file_path] = file_path.value
    params_struct[mpi_compile] = mpi_compile.value
    params_struct[mpi_type] = mpi_type.value
    the_script = f'#!/bin/bash \n#SBATCH -N {params_struct[num_nodes]} \n#SBATCH --ntasks-per-node={params_struct[num_tasks_per_node]} \nexport SHARED_FS_NAME=/mnt/orangefs  \nmodule add openmpi/3.0.0 \ncd $SHARED_FS_NAME/samplejobs/mpi \n{params_struct[mpi_compile]} mpi_prime.c -o mpi_prime -lm \n{params_struct[mpi_type]} -np {params_struct[num_nodes]*params_struct[num_tasks_per_node]} $SHARED_FS_NAME/samplejobs/mpi/mpi_prime\n'        
    return the_script
     

In [8]:
theScript = createScript(params)

In [9]:
def submitJob(b):
    try:
        output = client.ccqsub(num_nodes.value, num_tasks_per_node.value, '/mnt/orangefs/samplejobs/mpi/','testJob',job_body=txtArea.value)
        #pprint (output)
        pprint.pprint(output)
    except urllib.request.URLError as err:
            print("URL error: {0}".format(err))

In [10]:
if (ccq_menu.value) == 'ccqstat':
    try:
        output = client.ccqstat()
        #pprint (output)
        pprint.pprint(output)
    except urllib.request.URLError as err:
            print("URL error: {0}".format(err))
if (ccq_menu.value) == 'ccqsub':
    print('Job Script:')
    txtArea = widgets.Textarea(
        value=createScript(params),
        placeholder='Job Script',
        description='',
        disabled=False
    )
    btn = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Submit',
        icon=''
    )
    lbl3=widgets.Label() 
    display(txtArea,lbl3, btn)
    btn.on_click(submitJob)
    
if (ccq_menu.value) == 'ccqdel':
    output = client.ccqdel("247153")
    pprint.pprint(output)
    
  




Job Script:


Textarea(value='#!/bin/bash \n#SBATCH -N 2 \n#SBATCH --ntasks-per-node=2 \nexport SHARED_FS_NAME=/mnt/orangefs…

Label(value='')

Button(description='Submit', style=ButtonStyle(), tooltip='Submit')